# Import

In [1]:
import re
import os
import nltk
import string
from pprint import pprint

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd

In [4]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/ipynb')
os.listdir()

['mood_WS.ipynb', 'data.zip', 'data']

# Preprocessed

## Load Data

In [ ]:
# df_headline = pd.read_csv('../data/news/Anue_headline.csv')
df_wdstock = pd.read_csv('../data/news/Anue_wdstock.csv')

In [ ]:
# df_headline['CONTENT'] = df_headline['title']+'。'+df_headline['content']
df_wdstock['CONTENT'] = df_wdstock['title']+'。'+df_wdstock['content']

In [ ]:
# df_headline = df_headline[['publishAt', 'CONTENT']]
df_wdstock = df_wdstock[['publishAt', 'CONTENT']]

## Lower Casing

In [ ]:
# df_headline['CONTENT'] = df_headline['CONTENT'].str.lower()
df_wdstock['CONTENT'] = df_wdstock['CONTENT'].str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Removal of Punctuations

In [ ]:
PUNCT_TO_REMOVE = string.punctuation
print('PUNCT_TO_REMOVE ->', PUNCT_TO_REMOVE)

PUNCT_TO_REMOVE -> !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
df_headline['CONTENT'] = df_headline['CONTENT'].str.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
df_wdstock['CONTENT'] = df_wdstock['CONTENT'].str.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

## Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
def text_stemmer(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

In [ ]:
df_headline['CONTENT'] = df_headline['CONTENT'].apply(text_stemmer)
df_wdstock['CONTENT'] = df_wdstock['CONTENT'].apply(text_stemmer)

## Lemmatization

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [ ]:
def text_lemmatization(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:
df_headline['CONTENT'] = df_headline['CONTENT'].apply(text_lemmatization)
df_wdstock['CONTENT'] = df_wdstock['CONTENT'].apply(text_lemmatization)

In [ ]:
df_headline.to_excel('../data/headline_PREPROCESSED.xlsx', index=None)
df_wdstock.to_excel('../data/wdstock_PREPROCESSED.xlsx', index=None)

# CKIP WS

In [5]:
# df_headline = pd.read_excel('../data/headline_PREPROCESSED.xlsx')
df_wdstock = pd.read_excel('../data/wdstock_PREPROCESSED.xlsx')

In [6]:
!pip install ckiptagger

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from ckiptagger import data_utils, construct_dictionary, WS

In [ ]:
# data_utils.download_data_gdown("./")
ws = WS("./data")

Access denied with the following error:



 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771 



In [ ]:
df_dict = pd.read_excel('../data/dict/total_dict.xlsx')
df_dict = df_dict[df_dict.columns.drop('fund')]
dict_list = df_dict.values.tolist()
dict_list = [x for sublist in dict_list for x in sublist if str(x) != 'nan']

df_mood_dict = pd.read_excel('../data/dict/情緒字典_輸入程式.xlsx')
mood_dict_list = df_mood_dict['word'].tolist()

dict_list = dict_list + mood_dict_list

In [ ]:
dict_for_CKIP = dict((el, 1) for el in dict_list)
dict_for_CKIP = construct_dictionary(dict_for_CKIP)

In [ ]:
df_wdstock['CONTENT_CKIP'] = ws(df_wdstock['CONTENT'], coerce_dictionary=dict_for_CKIP)

In [ ]:
# df_headline.to_excel('../data/headline_PREPORCESSED.xlsx', index=None)
df_wdstock.to_excel('../data/wdstock_PREPORCESSED.xlsx', index=None)